In [1]:
import xarray as xr
import proplot as pplt
import numpy as np

In [ ]:
# Reads data (diurnal / semidiurnal) and plots the sum of all tidal components for u, v, w, T and rho
# Control which components by changing the lists

fig = pplt.figure(figwidth='8.3cm')
array = [  # the "picture" (0 == nothing, 1 == subplot A, 2 == subplot B, etc.)
    [1, 1, 2, 2],
    [3, 3, 4, 4],
    [0, 5, 5, 0],
]

ax = fig.subplots(array,proj='cyl')


ax.format(coast=True,landzorder=4,latlines=30, lonlines=60,abc=True, abcloc='ul')

wave="w1"
diurnal=True
variables=[f"{wave}_u",f"{wave}_v",f"{wave}_w",f"{wave}_t",f"{wave}_d"]
variables=["_u","_v","_w","_t","_d"]
variables_str=['u','v','w','T','rho']
diurnal_waves=['w1','w2','w3','w4','e1','e2','e3','s0']
#diurnal_waves=['w2'] # example
#diurnal_waves=['s0'] # example
urnal = ['diurnal','semidiurnal']
LST=17.5 # in hours
altitude=87.5 # in km
mon=2 # month

i = 0
datax = xr.Dataset()
for variable in variables:
    first=True

    for urnal_str in urnal:

        for wave in diurnal_waves:

            #if (wave == 'w1' and urnal_str == 'diurnal') or (wave == 'w2' and urnal_str == 'semidiurnal'):
                #print(f'SKIPPED! {wave} and {urnal_str}')
            #    b=5

            #else:

            lon=np.linspace(-np.pi,np.pi,360)

            diurnal = xr.open_dataset(f'ctmt_{urnal_str}_2002_2008.nc')
            diurnal = diurnal.sel(lev=altitude)
            diurnal = diurnal.sel(month=mon)

            phasevar=f"phase_{wave}{variable}"
            ampvar=f"amp_{wave}{variable}"

            try:

                amp = diurnal[ampvar]
                units = amp.units
                phase = diurnal[phasevar]

                amp = np.resize(diurnal[ampvar],(len(lon),len(diurnal[ampvar])))
                phase = np.resize(diurnal[phasevar],(len(lon),len(diurnal[phasevar])))
                lon = np.resize(lon, (len(diurnal[phasevar]),len(lon)))

                if wave[0] == "w":
                    s = - int(wave[1])
                    #print(f's = {s}')
                if wave[0] == "e":
                    s = int(wave[1])
                    #print(f's = {s}')
                
                if wave[0] == "s":
                    s = 0
                    #print(f's = {s}')

                if urnal_str == 'diurnal':
                    n = 1

                else:
                    n = 2

                if first:
                    data=amp.T*np.cos(n*2*np.pi/24*(LST-phase.T)-(s+n)*lon)
                    first=False
                else:
                    #print(wave)
                    data=data+amp.T*np.cos(n*2*np.pi/24*(LST-phase.T)-(s+n)*lon)


            except:
                #print(f'No {wave} in {urnal_str}')
                b=5

    datax[variable] = (('lat','lon'), data)
    datax[variable].attrs["units"] = units

    m=ax[i].contourf(lon[0,:]/np.pi*180,diurnal.lat,data,robust=98,extend='both',title=variable)
    ax[i].colorbar(m,loc='b',label=units)
    ax[i].format(title=variables_str[i])
    i = i + 1

datax['lon'] = lon[0,:]/np.pi*180
datax['lat'] =  diurnal.lat

fig.suptitle(f'Tidal fields at {altitude} km - February - LST {LST}')

fig.savefig(f'alt{altitude}_month{mon}_LST{LST}.png',format='png')
